In [ ]:
!wget https://downloads.dbpedia.org/repo/dbpedia/generic/redirects/2022.12.01/redirects_lang=en.ttl.bz2
!bunzip2 redirects_lang=en.ttl.bz2

In [ ]:
from redis import StrictRedis

redis_redir = StrictRedis(host='localhost', port=7979, db=1) # db=1 is redirects

In [ ]:
import rdflib

In [ ]:
NS_RESOURCE = 'http://dbpedia.org/resource/'
NS_RESOURCE_LEN = len(NS_RESOURCE)

In [ ]:
import sys
from rdflib import Graph, URIRef, BNode, Literal
from rdflib.plugins.parsers import ntriples
from rdflib.exceptions import ParserError as ParseError

class CustomSink(ntriples.NTGraphSink):
    def __init__(self, graph):
        super().__init__(graph)
        self.counter = 0
    def triple(self, s, p, o):
        source = str(s)[NS_RESOURCE_LEN:]
        target = str(o)[NS_RESOURCE_LEN:]
#         print((source, target))
        redis_redir.set(source, target)
        self.counter += 1
        if self.counter % 1_000_000 == 0:
            print(f"{int(self.counter / 1_000_000)} M")

class CustomParser(ntriples.W3CNTriplesParser):
    def parse(self, f):
        if not hasattr(f, "read"):
            raise ParseError("Item to parse must be a file-like object.")

        if not hasattr(f, "encoding") and not hasattr(f, "charbuffer"):
            # someone still using a bytestream here?
            f = codecs.getreader("utf-8")(f)

        self.file = f  # type: ignore[assignment]
        self.buffer = ""
        while True:
            self.line = self.readline()
            if self.line is None:
                break
            try:
                self.parseline()
            except ParseError:
                pass # ignore parse errors
        return self.sink

fname = r"redirects_lang=en.ttl"
sink = CustomSink(Graph())
parser = CustomParser(sink)
with open(fname, encoding='utf-8') as f:
    parser.parse(f)

In [ ]:
# redis_redir.flushdb()
redis_redir.save()